In [6]:
import requests
import os
import json

In [7]:
url = "https://annotations.allmaps.org/images/2e1d3f991aad6cb4"

In [8]:
df = requests.get(url).json()

opath = "data/input.json"

os.makedirs(os.path.dirname(opath), exist_ok=True)

with open(opath, "w") as f:
    json.dump(df, f, indent=2)

In [9]:
canvas_id = "http://www.example.org/georeferenced-canvas.json"
label = {
    "ja": [
        "東京帝國大學本部構内及農學部建物鳥瞰圖"
    ]
}

def convert_to_2(df):

    target = df["items"][0]["target"]

    service_id = target["service"][0]["@id"] + "/info.json"

    df_i = requests.get(service_id).json()

    opath_i = "data/input_info.json"

    os.makedirs(os.path.dirname(opath_i), exist_ok=True)

    with open(opath_i, "w") as f:
        json.dump(df_i, f, indent=2)

    width = df_i["width"]
    height = df_i["height"]

    features = []

    for feature in df["items"][0]["body"]["features"]:
        feature_new = {
            "type": "Feature",
            "properties": {
                "resourceCoords": feature["properties"]["pixelCoords"]
            },
            "geometry": {
                "type": "Point",
                "coordinates": feature["geometry"]["coordinates"]
            }
        }

        features.append(feature_new)

    res = {
        "@context": [
            "http://iiif.io/api/extension/georef/1/context.json",
            "http://iiif.io/api/presentation/3/context.json"
        ],
        "id": "http://www.example.org/canvas-annotation.json",
        "type": "Annotation",
        "motivation": "georeferencing",
        "target": {
            "id": "http://www.example.org/canvas.json",
            "type": "Canvas",
            "label": label,
            "height": height,
            "width": width,
            "items": [
            {
                "id": "http://www.example.org/contentPage.json",
                "type": "AnnotationPage",
                "items": [
                {
                    "id": "http://www.example.org/content.json",
                    "type": "Annotation",
                    "motivation": "painting",
                    "body": {
                    "id": f"{df_i['@id']}/full/full/0/default.jpg",
                    "type": "Image",
                    "format": "image/jpeg",
                    "height": height,
                    "width": width,
                    "service": [
                        {
                            "@id": df_i["@id"],
                            "type": "ImageService2"
                        }
                    ]
                    },
                    "target": "http://www.example.org/canvas.json"
                }
                ]
            }
            ]
        },
        "body": {
            "id": "http://www.example.org/feature-collection.json",
            "type": "FeatureCollection",
            "transformation": {
                "type": "polynomial",
                "options": {
                    "order": 1
                }
            },
            "features": features
        }
    }

    opath_r = "data/output.json"

    os.makedirs(os.path.dirname(opath_r), exist_ok=True)

    with open(opath_r, "w") as f:
        json.dump(res, f, indent=2)


In [10]:
convert_to_2(df)